In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [3]:
import warnings
warnings.filterwarnings("ignore")

Для начала загрузим наша датасет, которые использовался ранее в предыдущих работах. (https://www.kaggle.com/laotse/credit-risk-dataset)

In [4]:
df = pd.read_csv('credit_risk_dataset.csv')

Посмотрим на наш датасет, что бы убедиться, что все корретно считалось.

In [5]:
df.head(2)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2


In [6]:
df.tail(2)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,N,26
32580,66,42000,RENT,2.0,MEDICAL,B,6475,9.99,0,0.15,N,30


Все хорошо, продолжаем работу.

Так как наш датасет связан с кредитным скорингом, то мы будем предсказывать вероятность принадлежность к одному из классов {0, 1} нашего целевого признака **loan_status**, где 1 обозначает очень сильную уверенность в том, что человеку можно выдать кредит.

Так как некоторые объекты в данных описываются категориальными признаками применин к ниму one-hot-encoding.

In [7]:
numerical_features = pd.DataFrame(df[df.select_dtypes(['int64', 'float64']).columns])
categorial_features = pd.DataFrame(pd.DataFrame(df[df.select_dtypes(['object']).columns]))

In [8]:
one_hot_encoding_features = pd.get_dummies(categorial_features)

In [9]:
one_hot_encoding_features

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_N,cb_person_default_on_file_Y
0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
4,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
32577,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
32578,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
32579,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0


Теперь добавим закодированные категориальные признаки к нашим числовым данных.

In [10]:
X = pd.DataFrame(np.hstack([numerical_features, one_hot_encoding_features]),
                 columns=list(numerical_features.columns) + list(one_hot_encoding_features.columns))

In [11]:
print(f'X shape: {X.shape}')

X shape: (32581, 27)


В итоге мы имеем 8 числовых признаков (среди которых один является таргетом), и 19 признаков которые появлись в следствии one-hot-encoding (можно заметить, что категориальные значения принимают довольно малое число уникальных значений).

Теперь отделим от данных основной таргет (признак **loan_status**).

In [12]:
target = X['loan_status']

In [13]:
X = X.drop(columns=['loan_status'])

Тестировать мы будем логистическую регресиию, которая представлена в sklearn классом LogisticRegression.

Сразу следует обозначить следующую проблему - это масштаб и смещение признаков. Это заставляется хуже работать логистическую регрессию, так как она использует регуляризацию, то есть мы штрафуем модель за большие веса для признаков, тем самым заставляя их быть как можно меньше, а различный масштаб признаков плохо сочетается с этим. (то есть может возникнуть ситуация когда мы выбираем более далекое от правильно решения, но с маленькими весами, вместо решения у которого некоторые веса могут быть достаточно большими (из-за естественной потребности, потому что масштаб разный), но которое довольно близко к настоящему решению).


Поэтому по причиве выше нам необходимо нормировать наши данные, а уже потом тестировать различные комбинации которые предлагаются в лабораторной работе. (заполнять или не заполнять пропуски, сглаживать или не сглаживать данные).

Далее что бы получить более "хорошие" данные (тут более "хорошие" необходимо понимать в относительном смысле, потому что гиперпараметры мы будем подбирать при фиксированной метрике roc_auc и на одной конректной параметризации входных данных, но в среднем стоит ожидать улучшения результатов), сначала подберем некоторые гиперпараметры нашей модели. Так как sklearn по умолчанию использую L2 регуляризацию, необходимо подбирать коэффициент C, который отвечает за "силу" регуляризации. Но кроме этого подберем еще коэффициент l1_ration, которые отвечает за долю смешания L1 ти L2 регуляризаций, то есть в конечном итоге получим такую сместь l1_ratio * L1 + (1 - l1_ration) * L2, то есть в итоге мы будем использовать так называемую elasticnet регуляризацию.

Подбирать гиперпараметры будем на данных из которых предварительно выкинем все объекты по которым имеются пропуски, после этого отнормируем данные и будем подбирать гиперпараметры на кросс-валидации с 5 фолдами. А потом имею уже в некоторой мере "хорошые" гиперпараметры для нашей логистической регрессии начнем основное тестирование.

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

In [15]:
params_grid = {
    "C" : np.linspace(0.001, 100, 30),
    "l1_ratio" : np.linspace(0, 1, 10)
}

In [16]:
grid_cv = GridSearchCV(LogisticRegression(penalty='elasticnet', solver='saga'), 
                       params_grid, cv=KFold(5), scoring='roc_auc', n_jobs=-1)

Теперь подготовим данные для обучения на кросс-валидации.

In [17]:
data = pd.DataFrame(np.hstack([X, target[:, np.newaxis]]), columns=list(X.columns) + ['loan_status'])
data = data.dropna()

In [18]:
X_local = data[data.columns[: -1]]
target_local = data[data.columns[-1]]

In [19]:
scaler = StandardScaler()
X_local = scaler.fit_transform(X_local)

Теперь начинаем перебор по сетке.

In [20]:
grid_cv.fit(X_local, target_local)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='elasticnet',
                                          random_state=None, solver='saga',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated...
       6.89658276e+01, 7.24140690e+01, 7.58623103e+01, 7.93105517e+01,
       8.27587931e+01, 8.62070345e+01, 8.96552759e+01, 9.31035172e+01,
       9.65517586e+01, 1.00000000e+02]),
                         'l1_ratio': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       

In [21]:
print(f'best score {grid_cv.best_score_} on {grid_cv.best_params_}')

best score 0.8679366581480856 on {'C': 3.4492413793103447, 'l1_ratio': 1.0}


Отлично, получили неплохие результаты относительно метрики roc_auc. Далее продолжим работать с логистической регрессией с гиперпараметрами C = 3.449241 и l1_ration = 2/3.

Теперь начнем наше тестирования, для этого напишем необходимые функции. Для заполнения данных будем использовать SimpleImputer(strategy='mean') (можно было бы попробовать использовать KNNImputer, но он работает за квадратичное время от количества объектов, а их у нас много...) и для сглаживание данных - экспоненциальное сглаживание.

В качестве метрик будем использовать 'roc-auc', 'f1-macro', 'accuracy', 'precision' - классический выбор для задач классификации.

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score
from sklearn.impute import KNNImputer, SimpleImputer
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing

In [23]:
def smooth_data(values):
    fit_data = SimpleExpSmoothing(values, initialization_method="estimated").\
                fit(smoothing_level=0.2,optimized=False)
    return fit_data.fittedvalues

In [24]:
def dropna_numpy(x):
    original_shape = x.shape
    return x[~np.isnan(x)].reshape(original_shape)

In [25]:
def dropna_dataset(X, y):
    train_full = np.hstack([X, y[:, np.newaxis]])
    train_full = pd.DataFrame(train_full).dropna().to_numpy()
    X_train = train_full[:, :-1]
    y_train = train_full[:, -1]
    return X_train, y_train

In [ ]:
smooth_correspond_y_target = pd.concat([X, target], axis=1).dropna()['loan_status']
smooth_data_with_drop_na = X.dropna().apply(smooth_data, axis=0)
simple_imputer = SimpleImputer(strategy='mean')
smooth_data_with_fill_na = pd.DataFrame(simple_imputer.fit_transform(X)).apply(smooth_data, axis=0)

In [27]:
def test_classifier(clf, X, target, train_size, fill_na, smooth, metric_calc, metric_name, output_result,
                    smooth_data_with_drop_na, smooth_correspond_y_target, smooth_data_with_fill_na):
    if smooth and fill_na:
        X = smooth_data_with_fill_na
    elif smooth and not fill_na:
        X = smooth_data_with_drop_na
        target = smooth_correspond_y_target
    
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=1-train_size, random_state=42)
    
    if fill_na:
        knn_imputer = SimpleImputer(strategy='mean')
        knn_imputer.fit(X_train)
        X_train = knn_imputer.transform(X_train)
        X_test = knn_imputer.transform(X_test)
    else:
        X_train, y_train = dropna_dataset(X_train, y_train)
        X_test, y_test = dropna_dataset(X_test, y_test)
       
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    clf.fit(X_train, y_train)
    predict_prob = clf.predict_proba(X_test)[:, 1]
    if metric_name == 'auc_roc':
        score = metric_calc(y_test, predict_prob)
    else:
        score = metric_calc(y_test, clf.predict(X_test))
    
    if fill_na and smooth:
        index_name = 'fill_na & smooth'
    elif not fill_na and smooth:
        index_name = 'non fill_na & smooth'
    elif fill_na and not smooth:
        index_name = 'fill_na & non smooth'
    else:
        index_name = 'non fill_na & non smooth'
    
    
    output_result.loc[index_name, metric_name] = score

Сделаем датафрейм для вывода результатов.

In [28]:
index_output = list([f'{np.round(train_size, 1)}/{np.round(1 - train_size, 1)}', 
                            'non fill_na & smooth', 'fill_na & smooth', 
                            'non fill_na & non smooth', 'fill_na & non smooth']
                          for train_size in [0.5, 0.6, 0.7, 0.8, 0.9])

In [29]:
columns_output = ['roc-auc', 'f1-macro', 'accuracy', 'precision']

In [30]:
metrics = {
    'roc-auc': roc_auc_score,
    'f1-macro': f1_score,
    'accuracy': accuracy_score,
    'precision': precision_score
}

In [31]:
output_results = []

In [32]:
for iter_idx, train_size in enumerate([0.5, 0.6, 0.7, 0.8, 0.9]):
    local_results = pd.DataFrame(columns=columns_output, index=index_output[iter_idx])
    for fill_na in [True, False]:
        for smooth in [True, False]:
            for metric_name in ['roc-auc', 'f1-macro', 'accuracy', 'precision']:
                test_classifier(LogisticRegression(penalty='elasticnet', solver='saga', 
                                                  **grid_cv.best_params_),
                                X, target, train_size, fill_na, smooth, metrics[metric_name], metric_name,
                                local_results, smooth_data_with_drop_na, 
                                smooth_correspond_y_target, smooth_data_with_fill_na)
    output_results.append(local_results)

In [33]:
results = pd.concat([output_results[idx] for idx in range(len(output_results))], axis=0)

В конечном итоге имеем следующие результаты:

In [34]:
results

,roc-auc,f1-macro,accuracy,precision
0.5/0.5,NaN,NaN,NaN,NaN
non fill_na & smooth,0.528338,0.119802,0.788603,0.668831
fill_na & smooth,0.526787,0.112545,0.788963,0.689873
non fill_na & non smooth,0.754907,0.645221,0.866452,0.766343
fill_na & non smooth,0.754731,0.644972,0.866061,0.765842
0.6/0.4,NaN,NaN,NaN,NaN
non fill_na & smooth,0.528925,0.123681,0.789717,0.641509
fill_na & smooth,0.525535,0.108962,0.787923,0.667984
non fill_na & non smooth,0.759119,0.652023,0.868317,0.770071
fill_na & non smooth,0.758266,0.650607,0.867567,0.768532


Можно сделать следующий вывод, выбор пропорции разбиения — компромисс. Действительно, большой размер обучения ведет к более качественным алгоритмам, но бОльшему шуму при оценке модели на тесте. И наоборот, большой размер тестовой выборки ведет к менее шумной оценке качества, однако обученные модели получаются менее точными

Также мы могли столкнуться с насыщение нашего алгоритма, когда добавление новых наблюдений в выборке не добавляет качества, и отчасти могли столкнуться с переобучением.

Экспоненциальное сглаживание по всем столбцам в целом ухудшало результат, это можно объяснить тем, что мы теряли нужную нам информацию для обучения или "отходя" дальше от линейного вида зависимости в пространстве признаков, и также природой нашей данных, которые не имееют в себе временной составляющей.